# Lecture 2: Operations
## Overview
1. Basic operations
2. Tensor types
3. Project speed dating
4. Placeholders and feeding inputs
5. Lazy loading


## Fun with TensorBoard

In [3]:
import tensorflow as tf
a = tf.constant(2)
b = tf.constant(3)
x = tf.add(a, b)
with tf.Session() as sess:
    # add this line to use TensorBoard.
    writer = tf.summary.FileWriter('./graphs', sess.graph) #Create the summary writer after graph definition and before running your session
    print(sess.run(x))
writer.close() # close the writer when you’re done using it

5


Run it:
- Go to terminal, run:

    ```
    python [yourprogram].py

    tensorboard --logdir="./graphs" --port 6006
    ```
        
- open browser and go to: http://localhost:6006/

Custom names in the graph:

In [4]:
import tensorflow as tf
a = tf.constant(2, name="a")
b = tf.constant(3, name="b")
x = tf.add(a, b, name="add")
with tf.Session() as sess:
    writer = tf.summary.FileWriter("./graphs", sess.graph)
    print(sess.run(x)) # >> 5
writer.close()

5


## More constants


`tf.constant(value, dtype=None, shape=None, name='Const', verify_shape=False)`
### 1.  create a constant: (similar to numpy)

In [1]:
import tensorflow as tf
a = tf.constant([2, 2], name="a")
b = tf.constant([[0, 1], [2, 3]], name="b")
x = tf.add(a, b, name="add")
y = tf.multiply(a, b, name="mul")
with tf.Session() as sess:
    x, y = sess.run([x, y])
print(x, y)
# >> [5 8] [6 12]

[[2 3]
 [4 5]] [[0 2]
 [4 6]]


### 2. tensors filled with a specific value:

1. `tf.zeros(shape, dtype=tf.float32, name=None)`
    - ~ `numpy.zeros`
    - creates a tensor of shape and all elements will be zeros (when ran in session)
    - more compact than other constants in the graph def → faster startup (esp.in distributed)
1. `tf.zeros_like(input_tensor, dtype=None, name=None, optimize=True)`
    - ~ `numpy.zeros_like`
    - creates a tensor of shape and type (unless type is specified) as the input_tensor but all elements are zeros
1. `tf.ones(shape, dtype=tf.float32, name=None)`
1. `tf.ones_like(input_tensor, dtype=None, name=None, optimize=True)`
1. `tf.fill(dims, value, name=None) `

In [14]:
x = tf.zeros([2, 3], tf.int32)  
y = tf.zeros_like(x)
z = tf.fill([2,3],8)
with tf.Session() as sess:
    x,y,z = sess.run([x,y,z])
print('x:\n{} \ny:\n{} \nz:\n{}'.format(x,y,z))



x:
[[0 0 0]
 [0 0 0]] 
y:
[[0 0 0]
 [0 0 0]] 
z:
[[8 8 8]
 [8 8 8]]


### 3. constants as sequences
1. tf.linspace(start, stop, num, name=None)
    - slightly different from np.linspace
2. tf.range(start, limit=None, delta=1, dtype=None, name='range')  
ensor objects are **not iterable**
`for _ in tf.range(4): # TypeError`

In [15]:
x = tf.linspace(10.0, 13.0, 4)
y = tf.range(3,18,3) # start = 3, stop = 18, delta = 3
z = tf.range(5) # limit = 5
with tf.Session() as sess:
    x,y,z = sess.run([x,y,z])
print('x:\n{} \ny:\n{} \nz:\n{}'.format(x,y,z))

x:
[ 10.  11.  12.  13.] 
y:
[ 3  6  9 12 15] 
z:
[0 1 2 3 4]


### 4. Randomly Generated Constants
1. `tf.random_normal(shape, mean=0.0, stddev=1.0, dtype=tf.float32, seed=None, name=None)`
2. `tf.truncated_normal(shape, mean=0.0, stddev=1.0, dtype=tf.float32, seed=None, name=None)`
3. `tf.random_uniform(shape, minval=0, maxval=None, dtype=tf.float32, seed=None, name=None)`
4. `tf.random_shuffle(value, seed=None, name=None)`
5. `tf.random_crop(value, size, seed=None, name=None)`
6. `tf.multinomial(logits, num_samples, seed=None, name=None)`
7. `tf.random_gamma(shape, alpha, beta=None, dtype=tf.float32, seed=None, name=None)`

8. `tf.set_random_seed(seed)`

## Operations
![Table from “Fundamental of Deep Learning”](figures/02_01.png)


In [17]:
a = tf.constant([3, 6])
b = tf.constant([2, 2])
# tf.add(a, b) # >> [5 8]
# tf.add_n([a, b, b]) # >> [7 10]. Equivalent to a + b + b
# tf.multiply(a, b) # >> [6 12] because mul is element wise
# tf.matmul(a, b) # >> ValueError
# tf.matmul(tf.reshape(a, [1, 2]), tf.reshape(b, [2, 1])) # >> [[18]]
# tf.div(a, b) # >> [1 3]
# tf.mod(a, b) # >> [1 0]

## TensorFlow Data Types
TensorFlow takes Python natives types: boolean, numeric (int, float), strings
### 0-d tensor ("scalar")


In [18]:
t_0 = 19
tf.zeros_like(t_0) # ==> 0
tf.ones_like(t_0) # ==> 1

<tf.Tensor 'ones_like:0' shape=() dtype=int32>

### 1-d tensor ("vector")

In [ ]:
t_1 = ['apple', 'peach', 'banana']
tf.zeros_like(t_1) # ==> ['' '' '']
tf.ones_like(t_1) # ==> TypeError: Expected string, got 1 of type 'int' instead.

### 2x2 tensor ("matrix")

In [ ]:
t_2 = [[True, False, False],
[False, False, True],
[False, True, False]]
tf.zeros_like(t_2) # ==> 2x2 tensor, all elements are False
tf.ones_like(t_2) # ==> 2x2 tensor, all elements are True

![TensorFlow Data Types](figures/02_02.png)

### TF vs NP data types
- TensorFlow integrates seamlessly with NumPy
- Can pass numpy types to TensorFlow ops
- For `tf.Session.run(fetches)`: If the requested fetch is a Tensor , then the output will be a NumPy ndarray  
**do NOT use Python native types for tensors** because TensorFlow has to infer Python type

In [2]:
import tensorflow as tf
import numpy as np
tf.int32 == np.int32 # True
tf.ones([2, 2], np.float32) # ⇒ [[1.0 1.0], [1.0 1.0]]

<tf.Tensor 'ones:0' shape=(2, 2) dtype=float32>

## Variables
### 1. Problem about constants: stored in the graph definition
- makes loading graphs expensive when constants are big
- Only use constants for primitive types
- Use variables or readers for more data that requires more memory

In [4]:
import tensorflow as tf
my_const = tf.constant([1.0, 2.0], name="my_const")
with tf.Session() as sess:
    print(sess.graph.as_graph_def())
# you will see value of my_const stored in the graph’s definition

node {
  name: "ones"
  op: "Const"
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_FLOAT
        tensor_shape {
          dim {
            size: 2
          }
          dim {
            size: 2
          }
        }
        float_val: 1.0
      }
    }
  }
}
node {
  name: "my_const"
  op: "Const"
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_FLOAT
        tensor_shape {
          dim {
            size: 2
          }
        }
        tensor_content: "\000\000\200?\000\000\000@"
      }
    }
  }
}
versions {
  producer: 21
}



### 2. create variables

In [5]:
# create variable a with scalar value
a = tf.Variable(2, name="scalar")
# create variable b as a vector
b = tf.Variable([2, 3], name="vector")
# create variable c as a 2x2 matrix
c = tf.Variable([[0, 1], [2, 3]], name="matrix")
# create variable W as 784 x 10 tensor, filled with zeros
W = tf.Variable(tf.zeros([784,10]))

`tf.Variable` not `tf.variable`  
`tf.constant` not `tf.Constant`  
- **`tf.Variable` is a class, but `tf.constant` is an op**
- tf.Variable holds several ops  
    - `x.initializer`:  init op
    - `x.value()`:  read op
    - `x.assign(...)`:  write op
    - `x.assign_add(...)`  and more
- have to **initialize** variables

In [6]:
# initializing all variables at once:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    
# Initialize only a subset of variables:
init_ab = tf.variables_initializer([a, b], name="init_ab")
with tf.Session() as sess:
    sess.run(init_ab)
    
# Initialize a single variable
W = tf.Variable(tf.zeros([784,10]))
with tf.Session() as sess:
    sess.run(W.initializer)

### 3. `eval()` a variable


In [13]:
# W is a random 700 x 100 variable object
W = tf.Variable(tf.truncated_normal([700, 10]))
with tf.Session() as sess:
    sess.run(W.initializer)
    print(W)

<tf.Variable 'Variable_5:0' shape=(700, 10) dtype=float32_ref>


In [14]:
with tf.Session() as sess:
    sess.run(W.initializer)
    print(W.eval())

[[ 0.10173247  0.41137847  0.60416692 ...,  0.61058724  1.54879153
   1.26241386]
 [ 0.19658576  1.73018301 -1.20196819 ...,  0.36177546  0.38009125
   0.32686082]
 [ 0.70562273  0.92717385 -1.74096    ..., -0.08008771 -0.0218105
  -0.44643229]
 ..., 
 [-0.95996022  0.41936466  1.64461589 ...,  0.50300205 -1.11956382
   0.40928677]
 [-0.49932933 -0.71199787  0.4709371  ...,  1.85623705  0.97819293
  -0.42091134]
 [ 1.40349829 -1.13628864  0.32201138 ...,  0.90177363  0.40511495
  -0.47675774]]


### 4. `tf.Variable.assign()`

In [15]:
W = tf.Variable(10)
W.assign(100)
with tf.Session() as sess:
    sess.run(W.initializer)
    print (W.eval()) 
# W.assign(100) doesn’t assign the value 100
# to W. It creates an assign op, and that op
# needs to be run to take effect.

10


In [16]:
W = tf.Variable(10)
assign_op = W.assign(100)
with tf.Session() as sess:
    sess.run(assign_op) # assign op did the initialization, no need to initialize variable
    print(W.eval())

100


**initializer op is the assign op** that assigns the variable’s initial value to the variable itself.

In [23]:
# create a variable whose original value is 2
my_var = tf.Variable(2, name="my_var")
# assign a * 2 to a and call that op a_times_two
my_var_times_two = my_var.assign(2 * my_var)
with tf.Session() as sess:
    sess.run(my_var.initializer)
    sess.run(my_var_times_two) # >> 4
    print(my_var.eval())

4


### 5. `assign_add()` and `assign_sub()`
can’t initialize the variable because need its original value

In [25]:
my_var = tf.Variable(10)
with tf.Session() as sess:
    sess.run(my_var.initializer)
    # increment by 10
    sess.run(my_var.assign_add(10)) # >> 20
    # decrement by 2
    sess.run(my_var.assign_sub(2)) # >> 18
    print(my_var.eval())

18


### 6. each session maintains its own copy of variable

In [27]:
W = tf.Variable(10)
sess1 = tf.Session()
sess2 = tf.Session()
sess1.run(W.initializer)
sess2.run(W.initializer)
print(sess1.run(W.assign_add(10)))
print(sess2.run(W.assign_sub(2)))
sess1.close()
sess2.close()

20
8


### 7. use a variable to initialize another variable

In [ ]:
# W is a random 700 x 100 tensor
W = tf.Variable(tf.truncated_normal([700, 10]))

U = tf.Variable(2 * W) # not so safe (but quite common)
U = tf.Variable(2 * W.intialized_value()) # ensure that W is initialized before its value is used to initialize U

## Sessions
### 1. `Session` vs `InteractiveSession`
an `InteractiveSession` makes itself the default

In [28]:
sess = tf.InteractiveSession()
a = tf.constant(5.0)
b = tf.constant(6.0)
c = a * b
# We can just use 'c.eval()' without specifying the context 'sess'
print(c.eval())
sess.close()

30.0


### 2. `tf.Graph.control_dependencies(control_inputs)`
defines which ops should be run first

In [29]:
## your graph g have 5 ops: a, b, c, d, e
# with g.control_dependencies([a, b, c]):
## 'd' and 'e' will only run after 'a', 'b', and 'c' have executed.
# d = ...
# e = …

## Placeholder
### 1. 2 phases of tf program
- Assemble a graph
    - Can assemble the graph first without knowing the values needed for computation (use placeholders)
- Use a session to execute operations in the graph  

### 2. why placeholders
can later supply their own data when they need to execute the computation  

### 3. `tf.placeholder(dtype, shape=None, name=None)`

In [3]:
# create a placeholder of type float 32-bit, shape is a vector of 3 elements
a = tf.placeholder(tf.float32, shape=[3])
# create a constant of type float 32-bit, shape is a vector of 3 elements
# shape=None is easy to construct graphs, but nightmarish for debugging
b = tf.constant([5, 5, 5], tf.float32)
# use the placeholder as you would a constant or a variable
c = a + b # Short for tf.add(a, b)
#with tf.Session() as sess:
#    print(sess.run(c)) # Error because a doesn’t have any value

In [4]:
# Feed the values to placeholders using a dictionary
with tf.Session() as sess:
# feed [1, 2, 3] to placeholder a via the dict {a: [1, 2, 3]}
# fetch value of c
    print(sess.run(c, {a: [1, 2, 3]})) # the tensor a is the key, not the string ‘a’

[ 6.  7.  8.]


### 4. feed multiple data points in
feed all the values in, one at a time

In [5]:
#with tf.Session() as sess:
#    for a_value in list_of_values_for_a:
#        print sess.run(c, {a: a_value})

`tf.Graph.is_feedable(tensor)`: true iff tensor is feedable.

### 5. Feeding values to TF ops

In [2]:
import tensorflow as tf
# create operations, tensors, etc (using the default graph)
a = tf.add(2, 5)
b = tf.multiply(a, 3)
with tf.Session() as sess:
    # define a dictionary that says to replace the value of 'a' with 15
    replace_dict = {a: 15}
    # Run the session, passing in 'replace_dict' as the value to 'feed_dict'
    sess.run(b, feed_dict=replace_dict) # returns 45


### 6.  trap of lazy loading
normal loading

In [ ]:
x = tf.Variable(10, name='x')
y = tf.Variable(20, name='y')
z = tf.add(x, y) # you create the node for add node before executing the graph
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    writer = tf.summary.FileWriter('./my_graph/l2', sess.graph)
    for _ in range(10):
        sess.run(z)
    writer.close()

lazy loading: Defer creating/initializing an object until it is needed

In [ ]:
x = tf.Variable(10, name='x')
y = tf.Variable(20, name='y')
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    writer = tf.summary.FileWriter('./my_graph/l2', sess.graph)
    for _ in range(10):
        sess.run(tf.add(x, y)) # someone decides to be clever to save one line of code
    writer.close()

difference: lazy loading will miss the node Add

- normal loading: Node “Add” added once to the graph definition  
    node {
        name: "Add"
        op: "Add"
        input: "x/read"
        input: "y/read"
        attr {
            key: "T"
            value {
                type: DT_INT32
            }
        }
    }

- lazy loading: Node “Add” added 10 times to the graph definition. for more loops, graph gets bloated, Slow to load, Expensive to pass around  
    node {
        name: "Add"
        op: "Add"
        ...
    }
    ...
    node {
        name: "Add_9"
        op: "Add"
        ...
    }
      
**solution**: 
- Separate definition of ops from computing/running ops
- Use Python property to ensure function is also loaded once the first time it is called